In [1]:
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy swifter
!pip install flow_vis

     |████████████████████████████████| 647 kB 6.0 MB/s 
     |████████████████████████████████| 280 kB 97.6 MB/s 
     |████████████████████████████████| 136 kB 91.8 MB/s 


In [2]:
import os
import glob
import numpy as np
import random
import pandas as pd
from time import sleep
import cv2 
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import mediapy as media
from google.colab.patches import cv2_imshow
import mediapy as media
import flow_vis
import swifter
from sklearn.metrics import classification_report


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Create target directory & all intermediate directories if don't exists
def create_dir(dirName):
  try:
      os.makedirs(dirName)    
      print("Directory " , dirName ,  " Created ")
  except FileExistsError:
      print("Directory " , dirName ,  " already exists")  
      

In [6]:
PATH_DATA = '/content/drive/MyDrive/ucf_experiments/data/Hockey/'

PATH_TEST_RESIZED = PATH_DATA + 'resized/'
create_dir(PATH_TEST_RESIZED)

Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/resized/  already exists


In [7]:
PATH_CROP = PATH_DATA + 'data_crop/'
PATH_BLUR = PATH_DATA + 'data_blur/'

PATH_CROP_TEST = PATH_CROP + 'test/'
PATH_BLUR_TEST = PATH_BLUR + 'test/'

PATH_CHECKPOINTS = PATH_DATA + 'data_checkpoints/'
PATH_CHECKPOINTS_TEST = PATH_CHECKPOINTS + 'test/'

create_dir(PATH_CROP)
create_dir(PATH_BLUR)
create_dir(PATH_CROP_TEST)
create_dir(PATH_BLUR_TEST)

create_dir(PATH_CHECKPOINTS)
create_dir(PATH_CHECKPOINTS_TEST)


# Pós Crop
PATH_CROP_TEST_RESIZED = PATH_CROP + 'test_resized/'
create_dir(PATH_CROP_TEST_RESIZED)

PATH_CHECKPOINTS_TEST_RESIZED = PATH_CHECKPOINTS + 'test_resized/'
create_dir(PATH_CHECKPOINTS_TEST_RESIZED)

Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_blur/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_blur/test/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_checkpoints/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_checkpoints/test/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test_resized/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_checkpoints/test_resized/  already exists


# Geração de Labels

In [ ]:
def get_label(string):
  if(string.split('/')[-1][:2] == 'fi'):
    return 1
  elif(string.split('/')[-1][:2] == 'no'):
    return 0

In [ ]:
hk_list = glob.glob(PATH_DATA + 'data/*.avi')
df_videos = pd.DataFrame()
df_videos = pd.DataFrame({'video_path':hk_list})
df_videos['label'] = df_videos.apply(lambda row : get_label(row['video_path']), axis = 1)
df_videos

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
2,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
3,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
4,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
...,...,...
995,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
996,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
997,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
998,/content/drive/MyDrive/ucf_experiments/data/Ho...,0


In [ ]:
df_videos.label.value_counts()

1    500
0    500
Name: label, dtype: int64

# Pre Processing

## Resized Videos

In [ ]:
# Normalização do dimensionamento do vídeo
def test_video_normalize(video_numpy):
  with tf.device('/gpu:0'):
    augs = ['resized']
    video_tensor = tf.constant(video_numpy)
    resized = tf.image.resize(video_numpy, size=(224, 224))
    stacked  = tf.stack([resized])

    return augs, stacked


def test_normalization(row):
  video_name = str(row['video_path'].split('/')[-1].replace('.avi', '')) +  '_' + str(row['label'])
  raw_video = media.read_video(row['video_path'])
  raw_video = media.to_float01(raw_video)

  augs, videos = test_video_normalize(raw_video)
  
  for idx, name in enumerate(augs):
    aug_video_name = video_name + '_' + name + '.mp4'
    aug_path = os.path.join(PATH_TEST_RESIZED, aug_video_name)

    media.write_video(aug_path, videos[idx].numpy())

    row[name] = aug_path

  return row

In [ ]:
x_test_resized = df_videos.swifter.apply(test_normalization, axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
from pathlib import Path
paths = sorted(Path(PATH_TEST_RESIZED).iterdir(), key=os.path.getmtime, reverse=True)
len(paths)

1000

In [ ]:
print(len(glob.glob(PATH_TEST_RESIZED + '*.mp4')))

1000


In [ ]:
x_test_resized.head(3)

,video_path,label,resized
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...
2,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...


In [ ]:
x_test_resized.to_csv(
    PATH_TEST_RESIZED + 'x_test_resized.csv'
)

In [ ]:
from pathlib import Path
paths = sorted(Path(PATH_TEST_RESIZED).iterdir(), key=os.path.getmtime, reverse=True)
len(paths)

1001

## Optical Flow


In [ ]:
#videos_test = pd.read_csv(PATH_TEST_RESIZED + "x_test_resized.csv")
print(PATH_DATA)

/content/drive/MyDrive/ucf_experiments/data/Hockey/


In [ ]:

def flow_to_color(flow, hsv):
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def getOpticalFlow(video, return_rgb_flow=True):
    """Calculate dense optical flow of input video
    Args:
        video: the input video with shape of [frames,height,width,channel]. dtype=np.array
    Returns:
        flows_x: the optical flow at x-axis, with the shape of [frames,height,width,channel]
        flows_y: the optical flow at y-axis, with the shape of [frames,height,width,channel]
    """
    # initialize the list of optical flows
    
    gray_video = []
    for i in range(len(video)):
        img = cv2.cvtColor(video[i], cv2.COLOR_RGB2GRAY)
        gray_video.append(np.reshape(img,(224,224,1)))

    flows = []
    rgb_flows = []
    for i in range(0,len(video)-1):
        # calculate optical flow between each pair of frames


        flow = cv2.calcOpticalFlowFarneback(gray_video[i], gray_video[i+1], None, 0.5, 3, 15, 3, 5, 1.2, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        # subtract the mean in order to eliminate the movement of camera
        flow[..., 0] -= np.mean(flow[..., 0])
        flow[..., 1] -= np.mean(flow[..., 1])
        # normalize each component in optical flow
        flow[..., 0] = cv2.normalize(flow[..., 0],None,0,255,cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1],None,0,255,cv2.NORM_MINMAX)
        # Add into list 

        gss_app = cv2.GaussianBlur(flow,(5,5),1)

        if return_rgb_flow:
          flow_color = flow_vis.flow_to_color(gss_app, convert_to_bgr=False)
          rgb_flows.append(flow_color)

        flows.append(gss_app)
        
    # Padding the last frame as empty array
    flows.append(np.zeros((224,224,2)))

    flows = normalize(flows)
      
    return (np.array(flows, dtype=np.float32), np.array(rgb_flows))

def normalize(data):
    mean = np.mean(data)
    std = np.std(data)
    return (data-mean) / std

def dynamic_crop(opt_flows, rgb_frames):

    rgb_video = rgb_frames.copy()
    # sum of optical flow magnitude of individual frame
    magnitude = np.sum(opt_flows, axis=0)
    # filter slight noise by threshold 
    thresh = np.mean(magnitude)
    magnitude[magnitude<thresh] = 0
    # calculate center of gravity of magnitude map and adding 0.001 to avoid empty value
    x_pdf = np.sum(magnitude, axis=1) + 0.001
    y_pdf = np.sum(magnitude, axis=0) + 0.001
    # normalize PDF of x and y so that the sum of probs = 1
    x_pdf /= np.sum(x_pdf)
    y_pdf /= np.sum(y_pdf)

    x_pdf = x_pdf.sum(axis=1)
    y_pdf = y_pdf.sum(axis=1)

    # randomly choose some candidates for x and y 
    x_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)
    y_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=y_pdf)
    # get the mean of x and y coordinates for better robustness
    x = int(np.mean(x_points))
    y = int(np.mean(y_points))
    # avoid to beyond boundaries of array
    x = max(56,min(x,167))
    y = max(56,min(y,167))
    # get cropped video

    cropped = rgb_video[:,x-56:x+56,y-56:y+56,:].copy()

    w = 112
    h = 112

    start_point = (int(x-w/2), int(y-h/2))
    end_point = (int(x+w/2), int(y+h/2))

    color = (255,255,255)
    thickness = -1
    blurred_video = np.zeros_like(rgb_video)

    for idx, frame in enumerate(rgb_video):
      blurred_img = cv2.GaussianBlur(frame, (21, 21), 0)
      mask = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
      mask = cv2.rectangle(mask, start_point, end_point, color, thickness)

      best_cut = np.where(mask==np.array([255, 255, 255]), frame, blurred_img)
      blurred_video[idx] = best_cut

    return cropped, blurred_video

### Teste Visual

In [ ]:
videos_test = pd.read_csv(PATH_TEST_RESIZED + "x_test_resized.csv")
videos_test = videos_test[['resized', 'label']].rename(columns={'resized': 'video_path'})
videos_test

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
2,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
3,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
4,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
...,...,...
995,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
996,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
997,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
998,/content/drive/MyDrive/ucf_experiments/data/Ho...,0


In [ ]:

sample_url = videos_test.sample().video_path.squeeze()
sample_video = media.read_video(sample_url)

opt_frames, rgb_opt_frames = getOpticalFlow(sample_video, return_rgb_flow=True)

sample_url

'/content/drive/MyDrive/ucf_experiments/data/Hockey/resized/no363_xvid_0_resized.mp4'

In [ ]:
rgb_cut, rgb_blur = dynamic_crop(opt_frames, sample_video)

In [ ]:
media.show_videos([sample_video, rgb_opt_frames, rgb_blur, rgb_cut])

This browser does not support the video tag.,This browser does not support the video tag.,This browser does not support the video tag.,This browser does not support the video tag.


## Spliting Optical Flow (Crop + Blur)

In [ ]:
#videos_teste = pd.read_csv(PATH_CROP + 'test.txt', names=['path', 'label'], sep=';')
#splitted = np.array_split(videos_teste, 10)
print(videos_test.label.value_counts())

1    500
0    500
Name: label, dtype: int64


In [ ]:
splitted_test = np.array_split(videos_test, 10)
len(splitted_test)

10

In [ ]:
def process_videos(videos_df, save_blur_base, save_crop_base, only_humans=True):
  videos = []

  for idx, row in videos_df.iterrows():
    video_path = row['video_path']
    label = row['label']

    rgb_frames = media.read_video(video_path)
    opt_frames, rgb_opt_frames = getOpticalFlow(rgb_frames, return_rgb_flow=False)
    result_crop_frames = []
    result_blur_frames = []


    rgb_cut, rgb_blur = dynamic_crop(opt_frames, rgb_frames)

    if only_humans: 
      for rgb_frame, rgb_blur_frame in zip(rgb_cut, rgb_blur):
        pred = model(rgb_frame)
        detection_df  = pred.pandas().xyxy[0]
        has_person    = 'person' in detection_df.name.unique()
        if has_person:
          result_crop_frames.append(rgb_frame)
          result_blur_frames.append(rgb_blur_frame)
    else:
      result_crop_frames = rgb_cut
      result_blur_frames = rgb_blur

    if len(result_crop_frames) < 1:
      continue

    video_blur_name = video_path.split('/')[-1].split('.')[0] + '_blur.mp4'
    video_crop_name = video_path.split('/')[-1].split('.')[0] + '_crop.mp4'

    video_blur_output_path = os.path.join(save_blur_base, video_blur_name)
    video_crop_output_path = os.path.join(save_crop_base, video_crop_name)

    media.write_video(video_blur_output_path, np.array(result_blur_frames))
    media.write_video(video_crop_output_path, np.array(result_crop_frames))

    row = {
      'crop_path': video_crop_output_path,
      'blur_path': video_blur_output_path,
      'label': label
    }

    videos.append((video_blur_output_path, video_crop_output_path, label))

  return videos

In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0

for idx, split in tqdm(enumerate(splitted_test[start_idx:]), total=len(splitted_test) - start_idx):
  
  results = process_videos(
      split,
      save_blur_base=PATH_BLUR_TEST,
      save_crop_base=PATH_CROP_TEST,
      only_humans=False
  )

  checkpoints = checkpoints.append(results, ignore_index=True)
  checkpoints.to_csv(PATH_CHECKPOINTS_TEST+'{}_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [29:27<00:00, 176.72s/it]


In [ ]:
test_parts = glob.glob(PATH_CHECKPOINTS_TEST + '*_checkpoint.csv')
print(len(test_parts))

10


In [ ]:
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test

,Unnamed: 0,0,1,2
0,0,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
1,1,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
2,2,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
3,3,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
4,4,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
...,...,...,...,...
995,995,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
996,996,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
997,997,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
998,998,/content/drive/MyDrive/ucf_experiments/data/Ho...,/content/drive/MyDrive/ucf_experiments/data/Ho...,0


In [ ]:
"""def df_preprocessing(df):

  # Tratamento de dados
  #df = df.rename(columns={'new_label': 'label'})
  df = df.dropna()
  df = df.drop_duplicates()
  df['label'] = df.label.astype(int)

  return df

def processed_data_resized(data_files):
  processed_data = data_files.drop_duplicates()
  processed_data = processed_data.drop(columns=['Unnamed: 0'])

  processed_data = processed_data.dropna()
  processed_data = processed_data.reset_index(drop=True)

  dynamic_crop = processed_data[['new_path', 'label']]

  return dynamic_crop"""

In [ ]:
def processed_data(data_files):
  processed_data = data_files.drop_duplicates()
  processed_data = processed_data.drop(columns=['Unnamed: 0'])

  processed_data = processed_data.dropna()
  processed_data = processed_data.reset_index(drop=True)

  dynamic_blur = processed_data[['0', '2']]
  dynamic_crop = processed_data[['1', '2']]

  return dynamic_blur, dynamic_crop

In [ ]:
dynamic_blur_test, dynamic_crop_test = processed_data(files_test)
print(dynamic_blur_test.shape)
print(dynamic_crop_test.shape)

(1000, 2)
(1000, 2)


In [ ]:
dynamic_crop_test.head(2)['1'].to_list()

['/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test/fi105_xvid_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test/fi104_xvid_1_resized_crop.mp4']

In [ ]:
dynamic_crop_test.head(2)['2'].to_list()

[1, 1]

In [ ]:
dynamic_blur_test.to_csv(PATH_BLUR + 'test.txt', header=None, index=False, sep=';')
dynamic_crop_test.to_csv(PATH_CROP + 'test.txt', header=None, index=False, sep=';')

## Resized Pós Crop

**Apenas o Crop precisa desse processo, pois o blur já está nas dimensões corretas**

In [ ]:
# Load
videos_teste = pd.read_csv(PATH_CROP + 'test.txt', names=['path', 'label'], sep=';')
splitted = np.array_split(videos_teste, 10)
len(splitted)

10

In [ ]:
videos_teste.shape

(1000, 2)

In [ ]:
videos_teste.head(2)

,path,label
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1


In [ ]:
videos_teste.head(3).path.to_list()

['/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test/fi105_xvid_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test/fi104_xvid_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test/fi100_xvid_1_resized_crop.mp4']

In [ ]:
videos_teste.label.value_counts()

1    500
0    500
Name: label, dtype: int64

In [ ]:
def resized_videos(row):
  videos = []

  video_path = row['path']
  new_path = PATH_CROP_TEST_RESIZED + row['path'].split('/')[-1]

  rgb_frames = media.read_video(video_path)
  
  rgb_frames = media.to_float01(rgb_frames)

  sampled_video = tf.image.resize(rgb_frames, [224,224]).numpy()
  media.write_video(new_path, sampled_video)

  return new_path


checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.swifter.apply(resized_videos, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_CHECKPOINTS_TEST_RESIZED+'{}_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

  0%|          | 0/10 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

 10%|█         | 1/10 [01:27<13:09, 87.71s/it]

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

 20%|██        | 2/10 [02:48<11:08, 83.62s/it]

...

 80%|████████  | 8/10 [10:58<02:45, 82.60s/it]

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

 90%|█████████ | 9/10 [12:15<01:20, 80.88s/it]

Pandas Apply:   0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 10/10 [13:31<00:00, 81.15s/it]


In [ ]:
test_parts = glob.glob(PATH_CHECKPOINTS_TEST_RESIZED  +'*_checkpoint.csv')
print(len(test_parts))

10


In [ ]:
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test.shape

(1000, 4)

In [ ]:
files_test.head(3)

,Unnamed: 0,path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...
1,1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...
2,2,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...


In [ ]:
def processed_data_resized(data_files):
  processed_data = data_files.drop_duplicates()
  processed_data = processed_data.drop(columns=['Unnamed: 0'])

  processed_data = processed_data.dropna()
  processed_data = processed_data.reset_index(drop=True)

  dynamic_crop = processed_data[['new_path', 'label']]

  return dynamic_crop

In [ ]:
dynamic_crop_test_resized = processed_data_resized(files_test)
print(dynamic_crop_test_resized.shape)

(1000, 2)


In [ ]:
from pathlib import Path
paths = sorted(Path(PATH_CROP).iterdir(), key=os.path.getmtime, reverse=True)
paths

[PosixPath('/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test_resized'),
 PosixPath('/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test.txt'),
 PosixPath('/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test')]

In [ ]:
dynamic_crop_test_resized[['new_path', 'label']].to_csv(PATH_CROP + 'test_resized.txt', header=None, index=False, sep=';')

## Filtragem de quantidade mínima de frames

In [ ]:
#Blur
PATH_BLUR_TEST = PATH_BLUR + 'test.txt'

#Crop
PATH_CROP_TEST = PATH_CROP + 'test_resized.txt'

In [ ]:
def filter_small_videos(row):
  cap = cv2.VideoCapture(row['video_path'])
  totalframecount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  label = row['label']

  if totalframecount < 32:

    row['small'] = 1
  else:
    row['small'] = 0

  return row

tqdm.pandas()

### Blur 

In [ ]:
PATH_BLUR

'/content/drive/MyDrive/ucf_experiments/data/Hockey/data_blur/'

In [ ]:
videos_teste = pd.read_csv(PATH_BLUR + 'test.txt', sep=';', names=['video_path', 'label'])
print(videos_teste.shape)
print(videos_teste.label.value_counts())


(1000, 2)
1    500
0    500
Name: label, dtype: int64


In [ ]:
videos_teste = videos_teste.swifter.apply(filter_small_videos, axis=1)
videos_teste.to_csv(PATH_BLUR + 'test_vQtFrames.txt', header=None, index=False, sep=';')

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
df_videos_teste = pd.read_csv(PATH_BLUR + 'test_vQtFrames.txt', header=None, sep=';', names=['video_path', 'label', 'frames32'])
df_videos_teste.head()

,video_path,label,frames32
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0
2,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0
3,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0
4,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0


In [ ]:
df_videos_teste.label.value_counts()

1    500
0    500
Name: label, dtype: int64

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 1].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

ValueError: ignored

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 0].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

41
41
41
41
41
49
41
41
41
41


In [ ]:
df_videos_teste_filtered = df_videos_teste.loc[df_videos_teste.frames32 == 0]
df_videos_teste_filtered.shape

(1000, 3)

In [ ]:
df_videos_teste_filtered.label.value_counts()

1    500
0    500
Name: label, dtype: int64

In [ ]:
videos = {}
for key, video in df_videos_teste_filtered.sample(10).iterrows():
  videos[str(key)+'_'+str(video.label)] = media.read_video(video.video_path)

media.show_videos(videos)

744_0 This browser does not support the video tag.,188_1 This browser does not support the video tag.,122_1 This browser does not support the video tag.,916_0 This browser does not support the video tag.,709_0 This browser does not support the video tag.,180_1 This browser does not support the video tag.,598_0 This browser does not support the video tag.,186_1 This browser does not support the video tag.,349_1 This browser does not support the video tag.,110_1 This browser does not support the video tag.


In [ ]:
df_videos_teste_filtered[['video_path', 'label']].to_csv(PATH_BLUR + 'test_vFilter_min32frames.txt', header=None, index=False, sep=';')

### Crop

In [ ]:
PATH_CROP

'/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/'

In [ ]:
videos_teste = pd.read_csv(PATH_CROP + 'test_resized.txt', sep=';', names=['video_path', 'label'])
videos_teste

,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
2,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
3,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
4,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
...,...,...
995,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
996,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
997,/content/drive/MyDrive/ucf_experiments/data/Ho...,0
998,/content/drive/MyDrive/ucf_experiments/data/Ho...,0


In [ ]:
print(videos_teste.shape)
print(videos_teste.label.value_counts())

(1000, 3)
1    500
0    500
Name: label, dtype: int64


In [ ]:
videos_teste = pd.read_csv(PATH_CROP + 'test_resized.txt', sep=';', names=['video_path', 'label'])

videos_teste = videos_teste.swifter.apply(filter_small_videos, axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
videos_teste.to_csv(PATH_CROP + 'test_vQtFrames.txt', header=None, index=False, sep=';')

## Validando Teste

In [ ]:
df_videos_teste = pd.read_csv(PATH_CROP + 'test_vQtFrames.txt', header=None, sep=';', names=['video_path', 'label', 'frames32'])
df_videos_teste.head()

,video_path,label,frames32
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0
2,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0
3,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0
4,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,0


In [ ]:
df_videos_teste.head().video_path.tolist()

['/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test_resized/fi105_xvid_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test_resized/fi104_xvid_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test_resized/fi100_xvid_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test_resized/fi101_xvid_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/test_resized/fi106_xvid_1_resized_crop.mp4']

In [ ]:
df_videos_teste.label.value_counts()

1    500
0    500
Name: label, dtype: int64

In [ ]:
df_videos_teste.shape

(1000, 3)

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 1].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

ValueError: ignored

In [ ]:
videos = {}
for key, video in df_videos_teste.loc[df_videos_teste.frames32 == 0].sample(10).iterrows():
  cap = cv2.VideoCapture(video.video_path)
  print(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

41
41
41
41
41
41
41
41
41
41


In [ ]:
df_videos_teste_filtered = df_videos_teste.loc[df_videos_teste.frames32 == 0]
df_videos_teste_filtered.shape

(1000, 3)

In [ ]:
df_videos_teste_filtered.label.value_counts()

1    500
0    500
Name: label, dtype: int64

In [ ]:
videos = {}
for key, video in df_videos_teste_filtered.sample(10).iterrows():
  videos[str(key)+'_'+str(video.label)] = media.read_video(video.video_path)

media.show_videos(videos)

531_0 This browser does not support the video tag.,998_0 This browser does not support the video tag.,967_0 This browser does not support the video tag.,703_0 This browser does not support the video tag.,685_0 This browser does not support the video tag.,687_0 This browser does not support the video tag.,890_0 This browser does not support the video tag.,102_1 This browser does not support the video tag.,26_1 This browser does not support the video tag.,229_1 This browser does not support the video tag.


In [ ]:
df_videos_teste_filtered[['video_path', 'label']].to_csv(PATH_CROP + 'test_vFilter_min32frames.txt', header=None, index=False, sep=';')

## Uniform Sample

In [ ]:
#Blur
PATH_BLUR_UNIFORM_SAMPLE_TEST = PATH_BLUR + 'uniform_samples/test/'
create_dir(PATH_BLUR_UNIFORM_SAMPLE_TEST)
PATH_BLUR_UNIFORM_SAMPLE_CHECK_TEST = PATH_BLUR + 'uniform_samples/checkpoints_test/'
create_dir(PATH_BLUR_UNIFORM_SAMPLE_CHECK_TEST)

#Crop
PATH_CROP_UNIFORM_SAMPLE_TEST = PATH_CROP + 'uniform_samples/test/'
create_dir(PATH_CROP_UNIFORM_SAMPLE_TEST)
PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST = PATH_CROP + 'uniform_samples/checkpoints_test/'
create_dir(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST)

Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_blur/uniform_samples/test/  Created 
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_blur/uniform_samples/checkpoints_test/  Created 
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/uniform_samples/test/  Created 
Directory  /content/drive/MyDrive/ucf_experiments/data/Hockey/data_crop/uniform_samples/checkpoints_test/  Created 


### Blur

In [ ]:
videos_us_test = pd.read_csv(PATH_BLUR + 'test_vFilter_min32frames.txt', sep=';', header=None, names=['video_path', 'label'])

def uniform_sampling_test(row):
    path_save=PATH_BLUR_UNIFORM_SAMPLE_TEST
    target_frames=64
    # get total frames of input video and calculate sampling interval
    video_path = row['video_path']

    video = media.read_video(video_path)[...,:3]
    video = media.to_float01(video)

    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and 
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])     
    # calculate numer of padded frames and fix it 
    num_pad = target_frames - len(sampled_video)
    if num_pad>0:
        padding = [video[i] for i in range(-num_pad,0)]
        sampled_video += padding

            # get sampled video
    #sampled_video = tf.image.resize(sampled_video, [224,224])
    new_path = os.path.join(path_save, video_path.split('/')[-1])

    media.write_video(new_path, sampled_video)

    return new_path

In [ ]:
print(videos_us_test.shape)
display(videos_us_test.head(2))

(1000, 2)


,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1


In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

splitted = np.array_split(videos_us_test, 10)

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.apply(uniform_sampling_test, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_BLUR_UNIFORM_SAMPLE_CHECK_TEST+'{}_test_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [12:44<00:00, 76.41s/it]


In [ ]:
print(PATH_BLUR_UNIFORM_SAMPLE_TEST)

/content/drive/MyDrive/ucf_experiments/data/Hockey/data_blur/uniform_samples/test/


In [ ]:
test_parts = glob.glob(PATH_BLUR_UNIFORM_SAMPLE_CHECK_TEST + '*_checkpoint.csv')
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test.shape

NameError: ignored

In [ ]:
files_test.head(2)

,Unnamed: 0,video_path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...
1,1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...


In [ ]:
files_test[['new_path', 'label']].to_csv(PATH_BLUR + 'test_uniform_sample.txt', header=None, index=False, sep=';')

### Crop

In [ ]:
videos_us_test = pd.read_csv(PATH_CROP + 'test_vFilter_min32frames.txt', sep=';', header=None, names=['video_path', 'label'])

def uniform_sampling_test(row):
    path_save=PATH_CROP_UNIFORM_SAMPLE_TEST
    target_frames=64
    # get total frames of input video and calculate sampling interval
    video_path = row['video_path']

    video = media.read_video(video_path)[...,:3]
    video = media.to_float01(video)

    len_frames = int(len(video))
    interval = int(np.ceil(len_frames/target_frames))
    # init empty list for sampled video and 
    sampled_video = []
    for i in range(0,len_frames,interval):
        sampled_video.append(video[i])     
    # calculate numer of padded frames and fix it 
    num_pad = target_frames - len(sampled_video)
    if num_pad>0:
        padding = [video[i] for i in range(-num_pad,0)]
        sampled_video += padding

            # get sampled video
    #sampled_video = tf.image.resize(sampled_video, [224,224])
    new_path = os.path.join(path_save, video_path.split('/')[-1])

    media.write_video(new_path, sampled_video)

    return new_path

In [ ]:
print(videos_us_test.shape)
display(videos_us_test.head(2))

(1000, 2)


,video_path,label
0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1
1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1


In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

splitted = np.array_split(videos_us_test, 10)

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.apply(uniform_sampling_test, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST+'{}_test_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [14:14<00:00, 85.48s/it]


In [ ]:
test_parts = glob.glob(PATH_CROP_UNIFORM_SAMPLE_CHECK_TEST + '*_checkpoint.csv')
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test.shape

(1000, 4)

In [ ]:
files_test.head(2)

,Unnamed: 0,video_path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...
1,1,/content/drive/MyDrive/ucf_experiments/data/Ho...,1,/content/drive/MyDrive/ucf_experiments/data/Ho...


In [ ]:
files_test[['new_path', 'label']].to_csv(PATH_CROP + 'test_uniform_sample.txt', header=None, index=False, sep=';')

# Predições

## Imports

In [ ]:
import os
import glob
#from nvidia.dali import pipeline_def
#import nvidia.dali.fn as fn
#import nvidia.dali.types as types
#import nvidia.dali.plugin.tf as dali_tf
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf_v1
import logging
import numpy as np
#import cupy as cp
import mediapy
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import cv2
from time import sleep
from time import time

from tensorflow.keras import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Conv3D
from tensorflow.keras.layers import Reshape, AveragePooling2D 
from tensorflow.keras.layers import Dense, Dropout, Input 
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import LSTM, Multiply
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

from tensorflow import optimizers

tf.config.set_soft_device_placement(True)
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'false'
tf.get_logger().setLevel(logging.ERROR)


from keras.models import Sequential, Input, Model
from keras.models import load_model
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, Activation, LeakyReLU, Add, Multiply
from keras.regularizers import l2
from keras.layers.core import Lambda
from keras.layers.core import Lambda
from tensorflow.keras.optimizers import Adam, SGD

## Crop - Sem Mixed Precision

In [ ]:
max_batch_size = 5
NUM_WORKERS=8
EPOCHS=60
BATCH_SIZE=8

N_CLASSES = 2
IMSIZE = (224, 224)
SequenceLength = 64

##STEPS_PER_EPOCH = int(len(train_dataset_list) / BATCH_SIZE)
#VAL_STEPS_PER_EPOCH = int(len(test_dataset_list) / BATCH_SIZE)

#shapes = ((BATCH_SIZE, SequenceLength, IMSIZE[0], IMSIZE[1], 3), (BATCH_SIZE, N_CLASSES))
#dtypes = (tf.float32, tf.float32)

def create_model(continue_training=False, last_checkpoint=0):
  metrics=[
    tf.keras.metrics.Precision(),
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.AUC()
  ]

  inputs = Input(shape=(SequenceLength, IMSIZE[0], IMSIZE[1], 3))

  #####################################################
  rgb = inputs
  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  #####################################################
  x = MaxPooling3D(pool_size=(8,1,1))(rgb)

  #####################################################
  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,3,3))(x)

  #####################################################
  x = Flatten()(x)
  x = Dense(128,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(32, activation='relu')(x)
  pred = Dense(N_CLASSES, activation='softmax')(x)
  model = Model(inputs=inputs, outputs=pred)

  sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)

  return model

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.utils import Sequence
from keras.utils import np_utils
import mediapy as media

class DataGenerator(Sequence):
    """Data Generator inherited from keras.utils.Sequence
    Args: 
        directory: the path of data set, and each sub-folder will be assigned to one class
        batch_size: the number of data points in each batch
        shuffle: whether to shuffle the data per epoch
    Note:
        If you want to load file with other data format, please fix the method of "load_data" as you want
    """
    def __init__(self, input_file, batch_size=1, shuffle=True, data_augmentation=True, uniform_sample=False):
        # Initialize the params
        self.batch_size = batch_size
        self.input_file = input_file
        self.classes = 2
        self.shuffle = shuffle
        self.data_aug = data_augmentation
        self.uniform_sample = uniform_sample
        # Load all the save_path of files, and create a dictionary that save the pair of "data:label"
        self.X_path, self.Y_dict = self.search_data() 
        # Print basic statistics information
        self.print_stats()
        return None
        
    def search_data_v2(self):
        X_path = []
        Y_dict = {}

        df_videos = pd.read_csv(self.input_file, header=None, sep=';', names=['video_path', 'label'])

        X_path = df_videos[['video_path', 'label']].video_path.to_list()
        Y_dict = pd.get_dummies(df_videos[['video_path', 'label']], columns=['label']).set_index('video_path').T.to_dict(orient="list")

        return X_path, Y_dict

    def search_data(self):
        X_path = []
        Y_dict = {}
        one_hots = np_utils.to_categorical(range(self.classes))

        with open(self.input_file, 'r') as data_file:
           for line in data_file:
              line = line.rstrip('\n').split(';')
              path = line[0]

              label = int(line[1])
              X_path.append(path)
              Y_dict[path] = one_hots[label]

        return X_path, Y_dict
    
    def print_stats(self):
        # calculate basic information
        self.n_files = len(self.X_path)
        self.n_classes = self.classes
        self.indexes = np.arange(len(self.X_path))
        np.random.shuffle(self.indexes)
        # Output states
        print("Found {} files belonging to {} classes.".format(self.n_files,self.n_classes))
        for i,label in enumerate(range(self.classes)):
            print('%10s : '%(label),i)
        return None
    
    def __len__(self):
        # calculate the iterations of each epoch
        steps_per_epoch = np.ceil(len(self.X_path) / float(self.batch_size))
        return int(steps_per_epoch)

    def __getitem__(self, index):
        """Get the data of each batch
        """
        # get the indexs of each batch
        batch_indexs = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # using batch_indexs to get path of current batch
        batch_path = [self.X_path[k] for k in batch_indexs]
        # get batch data
        batch_x, batch_y = self.data_generation(batch_path)
        return batch_x, batch_y

    def on_epoch_end(self):
        # shuffle the data at each end of epoch
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def data_generation(self, batch_path):
        # load data into memory, you can change the np.load to any method you want
        batch_x = [self.load_data(x) for x in batch_path]
        batch_y = [self.Y_dict[x] for x in batch_path]
        # transfer the data format and take one-hot coding for labels
        batch_x = np.array(batch_x)
        batch_y = np.array(batch_y)
        return batch_x, batch_y
      
    def normalize(self, data):
        mean = np.mean(data)
        std = np.std(data)
        return (data-mean) / std
    
    def random_flip(self, video, prob):
        s = np.random.rand()
        if s < prob:
            video = np.flip(m=video, axis=2)
        return video    
    
    def uniform_sampling(self, video, target_frames=64):
        # get total frames of input video and calculate sampling interval 
        len_frames = int(len(video))
        interval = int(np.ceil(len_frames/target_frames))
        # init empty list for sampled video and 
        sampled_video = []
        for i in range(0,len_frames,interval):
            sampled_video.append(video[i])     
        # calculate numer of padded frames and fix it 
        num_pad = target_frames - len(sampled_video)
        if num_pad>0:
            padding = [video[i] for i in range(-num_pad,0)]
            sampled_video += padding     
        # get sampled video

        #sampled_video = tf.image.resize(sampled_video, [224,224])
        return np.array(sampled_video, dtype=np.float32)
    
    def dynamic_crop(self, video):
        # extract layer of optical flow from video
        opt_flows = video[...,3]
        # sum of optical flow magnitude of individual frame
        magnitude = np.sum(opt_flows, axis=0)
        # filter slight noise by threshold 
        thresh = np.mean(magnitude)
        magnitude[magnitude<thresh] = 0
        # calculate center of gravity of magnitude map and adding 0.001 to avoid empty value
        x_pdf = np.sum(magnitude, axis=1) + 0.001
        y_pdf = np.sum(magnitude, axis=0) + 0.001
        # normalize PDF of x and y so that the sum of probs = 1
        x_pdf /= np.sum(x_pdf)
        y_pdf /= np.sum(y_pdf)
        # randomly choose some candidates for x and y 
        x_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)
        y_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=y_pdf)
        # get the mean of x and y coordinates for better robustness
        x = int(np.mean(x_points))
        y = int(np.mean(y_points))
        # avoid to beyond boundaries of array
        x = max(56,min(x,167))
        y = max(56,min(y,167))
        # get cropped video 
        return video[:,x-56:x+56,y-56:y+56,:]  
    
    def color_jitter(self,video):
        # range of s-component: 0-1
        # range of v component: 0-255
        s_jitter = np.random.uniform(-0.2,0.2)
        v_jitter = np.random.uniform(-30,30)
        for i in range(len(video)):
            hsv = cv2.cvtColor(video[i], cv2.COLOR_RGB2HSV)
            s = hsv[...,1] + s_jitter
            v = hsv[...,2] + v_jitter
            s[s<0] = 0
            s[s>1] = 1
            v[v<0] = 0
            v[v>255] = 255
            hsv[...,1] = s
            hsv[...,2] = v
            video[i] = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
        return video
        
    def load_data(self, path):
        data = media.read_video(path)[...,:3]
        data = np.float32(data)
        # sampling 64 frames uniformly from the entire video

        if self.uniform_sample:
          data = self.uniform_sampling(video=data, target_frames=64)
        else:
          data = np.array(data, dtype=np.float32)

        # whether to utilize the data augmentation
        if  self.data_aug:
            data = self.color_jitter(data)
            data = self.random_flip(data, prob=0.5)
        # normalize

        data = self.normalize(data)
        return data

In [ ]:
df_hockey = DataGenerator(input_file=PATH_CROP + 'test_uniform_sample.txt',
                                batch_size=BATCH_SIZE, 
                                data_augmentation=False,
                                uniform_sample=False)
df_hockey

Found 1000 files belonging to 2 classes.
         0 :  0
         1 :  1


In [ ]:
def generate_predicts(data, model, path_weights, name_csv):
  df_results = pd.DataFrame()

  with tf.device("/gpu:0"):
    model = model
    model.load_weights(path_weights)
    
    for idx, payload in tqdm(enumerate(data)):

      items, labels = payload
      predictions = model.predict_on_batch(items)
      K.clear_session()

      batch = {
          'true':np.argmax(predictions, axis=1),
          'predictions':np.argmax(labels, axis=1)
      }

      _temp = pd.DataFrame(batch)

      df_results = df_results.append(_temp, ignore_index=True)
      if idx % 100 == 0:
        print(idx)
        
      if idx == 1000:
        break

  df_results.to_csv(PATH_DATA + name_csv)

  report = classification_report(y_pred = df_results['predictions'], y_true=df_results['true'])

  return df_results, report

In [ ]:
best_epoch = 30
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBaseline-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_hockey, create_model(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:19, 19.16s/it]

0


101it [09:31,  5.39s/it]

100


125it [11:41,  5.61s/it]


,true,predictions
0,1,0
1,0,1
2,0,0


              precision    recall  f1-score   support

           0       0.81      0.62      0.70       652
           1       0.50      0.72      0.59       348

    accuracy                           0.65      1000
   macro avg       0.65      0.67      0.65      1000
weighted avg       0.70      0.65      0.66      1000



### The Best Epoch

In [ ]:
best_epoch = 16
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBaseline-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_hockey, create_model(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:02,  2.99s/it]

0


101it [04:55,  2.91s/it]

100


125it [06:05,  2.93s/it]


,true,predictions
0,0,0
1,1,1
2,0,0


              precision    recall  f1-score   support

           0       0.79      0.75      0.77       524
           1       0.74      0.78      0.76       476

    accuracy                           0.76      1000
   macro avg       0.76      0.76      0.76      1000
weighted avg       0.77      0.76      0.76      1000



In [10]:
best_epoch = 16
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBaseline-epoch-{}.csv'.format(str(best_epoch))
df_results = pd.read_csv(PATH_DATA + name_csv)
report = classification_report(y_pred = df_results['predictions'], y_true=df_results['true'], digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.7880    0.7519    0.7695       524
           1     0.7400    0.7773    0.7582       476

    accuracy                         0.7640      1000
   macro avg     0.7640    0.7646    0.7639      1000
weighted avg     0.7652    0.7640    0.7641      1000



## Crop - Com Mixed Precision

In [ ]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [ ]:
max_batch_size = 5
NUM_WORKERS=8
EPOCHS=60
BATCH_SIZE=8

N_CLASSES = 2
IMSIZE = (224, 224)
SequenceLength = 64

def create_model2(continue_training=False, last_checkpoint=0):
  metrics=[
    tf.keras.metrics.Precision(),
    tf.keras.metrics.CategoricalAccuracy(),
    tf.keras.metrics.Recall(),
    tf.keras.metrics.AUC()
  ]

  inputs = Input(shape=(SequenceLength, IMSIZE[0], IMSIZE[1], 3))

  #####################################################
  rgb = inputs
  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      16, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      16, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  rgb = Conv3D(
      32, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = Conv3D(
      32, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(rgb)
  rgb = MaxPooling3D(pool_size=(1,2,2))(rgb)

  #####################################################
  x = MaxPooling3D(pool_size=(8,1,1))(rgb)

  #####################################################
  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      64, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      64, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,2,2))(x)

  x = Conv3D(
      128, kernel_size=(1,3,3), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = Conv3D(
      128, kernel_size=(3,1,1), strides=(1,1,1), kernel_initializer='he_normal', activation='relu', padding='same')(x)
  x = MaxPooling3D(pool_size=(2,3,3))(x)

  #####################################################
  x = Flatten()(x)
  x = Dense(128,activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(32, activation='relu')(x)
  pred = Dense(N_CLASSES, activation='softmax', dtype='float32')(x)
  model = Model(inputs=inputs, outputs=pred)

  sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=metrics)

  return model

In [ ]:
best_epoch = 30
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBaseline_mixed_precision-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline_mixed_precision - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_hockey, create_model2(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:03,  3.67s/it]

0


101it [04:58,  2.93s/it]

100


125it [06:08,  2.95s/it]


,true,predictions
0,0,0
1,1,1
2,0,0


              precision    recall  f1-score   support

           0       0.57      0.80      0.66       357
           1       0.85      0.66      0.75       643

    accuracy                           0.71      1000
   macro avg       0.71      0.73      0.70      1000
weighted avg       0.75      0.71      0.72      1000



### The Best Epoch

In [ ]:
best_epoch = 13
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBaseline_mixed_precision-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/model_vBaseline_mixed_precision - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_hockey, create_model2(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:03,  3.15s/it]

0


101it [04:56,  2.90s/it]

100


125it [06:06,  2.94s/it]


,true,predictions
0,0,0
1,0,1
2,0,0


              precision    recall  f1-score   support

           0       0.79      0.71      0.75       554
           1       0.68      0.77      0.72       446

    accuracy                           0.74      1000
   macro avg       0.74      0.74      0.74      1000
weighted avg       0.74      0.74      0.74      1000



In [11]:
best_epoch = 13
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBaseline_mixed_precision-epoch-{}.csv'.format(str(best_epoch))
df_results = pd.read_csv(PATH_DATA + name_csv)
report = classification_report(y_pred = df_results['predictions'], y_true=df_results['true'], digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.7920    0.7148    0.7514       554
           1     0.6840    0.7668    0.7230       446

    accuracy                         0.7380      1000
   macro avg     0.7380    0.7408    0.7372      1000
weighted avg     0.7438    0.7380    0.7388      1000



## Blur - Com Mixed Precision

In [ ]:
df_hockey = DataGenerator(input_file=PATH_BLUR + 'test_uniform_sample.txt',
                                batch_size=BATCH_SIZE, 
                                data_augmentation=False,
                                uniform_sample=False)
df_hockey

Found 1000 files belonging to 2 classes.
         0 :  0
         1 :  1


In [ ]:
best_epoch = 30
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBlur-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/model mixed precision - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_hockey, create_model2(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:15, 15.07s/it]

0


101it [09:23,  5.61s/it]

100


125it [11:36,  5.57s/it]


,true,predictions
0,1,1
1,0,0
2,1,1


              precision    recall  f1-score   support

           0       0.76      0.78      0.77       489
           1       0.78      0.76      0.77       511

    accuracy                           0.77      1000
   macro avg       0.77      0.77      0.77      1000
weighted avg       0.77      0.77      0.77      1000



### The Best Epoch

In [ ]:
best_epoch = 20
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBlur-epoch-{}.csv'.format(str(best_epoch))
path_weights = '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/model mixed precision - rwf/checkpoints/model.{}.h5'.format(str(best_epoch))
df_results, report = generate_predicts(df_hockey, create_model2(), path_weights, name_csv)
display(df_results.head(3))
print(report)

1it [00:02,  2.98s/it]

0


101it [04:53,  2.89s/it]

100


125it [06:03,  2.90s/it]


,true,predictions
0,1,1
1,0,0
2,0,1


              precision    recall  f1-score   support

           0       0.89      0.76      0.82       590
           1       0.71      0.87      0.78       410

    accuracy                           0.80      1000
   macro avg       0.80      0.81      0.80      1000
weighted avg       0.82      0.80      0.80      1000



In [12]:
best_epoch = 20
base = 'Hockey'
name_csv = 'Predicts-'+base+'-vBlur-epoch-{}.csv'.format(str(best_epoch))
df_results = pd.read_csv(PATH_DATA + name_csv)
report = classification_report(y_pred = df_results['predictions'], y_true=df_results['true'], digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8920    0.7559    0.8183       590
           1     0.7120    0.8683    0.7824       410

    accuracy                         0.8020      1000
   macro avg     0.8020    0.8121    0.8004      1000
weighted avg     0.8182    0.8020    0.8036      1000



# FIM

In [ ]:
FIM

## Load Data

In [ ]:
files_test.to_csv(PATH_BLUR + 'test_uniform_sample.txt', header=None, index=False, sep=';')video_path	label	new_path
files_test.to_csv(PATH_CROP + 'test_uniform_sample.txt', header=None, index=False, sep=';')




## Metrics

In [ ]:
validation_df = pd.DataFrame()

with tf.device("/gpu:0"):
  model = create_model()
  model.load_weights('/content/drive/MyDrive/ucf_experiments/experiment_3/data_crop/model/checkpoints/model.196.h5')
  
  for idx, payload in enumerate(test_dataset):


    items, labels = payload
    predictions = model.predict_on_batch(items)
    K.clear_session()

    batch = {
        'true':np.argmax(predictions, axis=1),
        'predictions':np.argmax(labels, axis=1)
    }

    _temp = pd.DataFrame(batch)

    validation_df = validation_df.append(_temp, ignore_index=True)
    if idx % 100 == 0:
      print(idx)
      
    if idx == 1000:
      break

validation_df.to_csv('/content/drive/MyDrive/ucf_experiments/experiment_3/data_crop/rwf_wy_dynamic_crop_conv3d_tf.csv')

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_pred = validation_df['predictions'], y_true=validation_df['true'])
print(report)